# Advanced RAG with OpenSearch neural plugin 
This lab will walk you an advanced RAG architecture where we will process a PDF document. We will first extract text from PDF, then we will chunk it using recursive character chunking technique. We will use OpenSearch neural plugin to help us convert these chunks into vector at the time of ingestion. Once data is ingested, we will use Neural plugin to semantically search the data. We will use the returned results to engineer a prompt to generate answers.

The PDF that we will is an annual report for 2023 published by Amazon. Within this document there is financial data, and there is performance, risks facing the company and guidance for future. We will be a financial analyst assistant bot that may answer questions from this document.

We will start by loading appropriate libraries.

## 1. Install pre-requisites and initialize variables.

In [ ]:
!pip install langchain langchain_community pypdf langchain_experimental --quiet
!pip install -qU langchain-text-splitters
!pip install --upgrade --quiet  boto3
!pip install pdfminer.six --quiet
!pip install opensearch-py --quiet
!pip install "unstructured[all-docs]" --quiet
!pip install pdf2image --quiet
!pip install -qU langchain-aws --quiet
!pip install alive_progress --quiet
!pip install opensearch-py-ml --quiet
!pip install requests_aws4auth --quiet

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain_community.chat_models import BedrockChat
from langchain.chains import ConversationalRetrievalChain

from langchain_community.embeddings import BedrockEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

import boto3
import os
import time
import json
import pandas as pd
from tqdm import tqdm
import sagemaker
from opensearchpy import OpenSearch, RequestsHttpConnection
from sagemaker import get_execution_role
import random 
import string
import s3fs
from urllib.parse import urlparse
from IPython.display import display, HTML
from alive_progress import alive_bar
from opensearch_py_ml.ml_commons import MLCommonClient
from requests_aws4auth import AWS4Auth
import requests 
import os
import json
import pandas as pd
import numpy as np

### Initializing variables from CloudFormation stack output

In [ ]:
# Create a Boto3 session
session = boto3.Session()

# Get the account id
account_id = boto3.client('sts').get_caller_identity().get('Account')

# Get the current region
region = session.region_name

cfn = boto3.client('cloudformation')
bedrock_client = boto3.client('bedrock-runtime')

# Method to obtain output variables from Cloudformation stack. 
def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

## Setup variables to for the rest of the demo
cloudformation_stack_name = "genai-data-foundation-workshop"

outputs = get_cfn_outputs(cloudformation_stack_name)
aos_host = outputs['OpenSearchDomainEndpoint']
s3_bucket = outputs['s3BucketTraining']
bedrock_inf_iam_role = outputs['BedrockBatchInferenceRole']
bedrock_inf_iam_role_arn = outputs['BedrockBatchInferenceRoleArn']
sagemaker_notebook_url = outputs['SageMakerNotebookURL']

# We will just print all the variables so you can easily copy if needed.
outputs

## 2. Extracting text from PDF and chunking
The most simplest way to chunk document would be by length, but keeping paragraphs or lines together so it does not lose the meaning. We will use langchain library's recursive character text splitter which offers ways to split data by length, yet keeps the lines, paragraph together as much as possible.


In [ ]:
# this method would split the text into chunks by paragraph, line boundary and keeping chunk 
# size as close to 1000 characters, it will also overlap the text between chunks if it were to 
# split line or paragraph in the middle.

def recursive_character_chunking(text): 
    text_splitter = RecursiveCharacterTextSplitter( #create a text splitter
        separators=["\n\n", "\n", ".", " "], #split chunks at (1) paragraph, (2) line, (3) sentence, or (4) word, in that order
        chunk_size=1000, #divide into 1000-character chunks using the separators above
        chunk_overlap=200, #number of characters that can overlap with previous chunk
        length_function=len,
        is_separator_regex=True,
    )
    
    docs = text_splitter.create_documents(text)#From the loaded PDF
    
    return docs #return the index to be cached by the client app

### Load and parse PDF file.
Now we will use Langchain library's PyPDFLoader to load our PDF and extract text

In [ ]:
#LOAD A PDF DOCUMENT

from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Amazon-com-Inc-2023-Annual-Report.pdf")
documents = loader.load()

#print(documents)a

texts = ""

for document in documents:
    texts += document.page_content.replace(r'\\n', '\n')

Now we will chunk the loaded PDF text using recursive character chunking technique

In [ ]:
#LETS RECURISIVE CHUNK IT
docs = recursive_character_chunking([texts])

# the method prints chunks
def print_chunks(data):
    #Let's print the chunks -- notice the overlap between chunk 3 and 4
    i = 1
    for doc in data:
        print(f"---------START OF CHUNK {i}------")
        print(f"{doc.page_content}")
        print(f"---------END OF CHUNK {i}------\n\n")
        i+=1

#Let's print first 5 chunks.
print_chunks(docs[:5])

## 3. Create a connection with OpenSearch domain.
Next, we'll use Python API to set up connection with OpenSearch domain.


##### NOTE: 
_At any point in this lab, if you get a failure message - **The security token included in the request is expired.**_ You can resolve it by running this cell again. The cell refreshes the security credentials that is required for the rest of the lab.

In [ ]:
kms = boto3.client('secretsmanager')
aos_credentials = json.loads(kms.get_secret_value(SecretId=outputs['DBSecret'])['SecretString'])

#credentials = boto3.Session().get_credentials()
#auth = AWSV4SignerAuth(credentials, region)
auth = (aos_credentials['username'], aos_credentials['password'])

aos_client = OpenSearch(
    hosts = [{'host': aos_host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)
ml_client = MLCommonClient(aos_client)
host = f'https://{aos_host}/'
service = 'es'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)
headers = {"Content-Type": "application/json"}



In [ ]:
#initializing some variables that we will use later.

connector_id = ""
model_id = ""

## 4. Create and deploy model connector to Amazon Bedrock Titan Text Embedding v2 

We are going to use Amazon Sagemaker Notebook IAM role to configure the connector in OpenSearch. This IAM Role has permission to pass BedrockInference IAM role to OpenSearch. OpenSearch will then be able to use BedrockInference IAM role to make calls to Bedrock models.

Following cell will create a connector using SageMaker Notebook IAM role. Following cell will create a OpenSearch remote model connector with Amazon Bedrock Titan Text embedding v2 model. Following cell defines the connector configuration.


#### Important pre-requisite
You should have followed the steps in the Lab instruction section to map Sagemaker notebook role to OpenSearch `ml_full_access` role. If not, please visit the lab instructions and complete the **Setting up permission for Notebook IAM Role** section. If you have not done this, you will get an error in the following cell.

In [ ]:
import boto3
import requests 
from requests_aws4auth import AWS4Auth
import json


if not connector_id:
    # Register repository
    path = '_plugins/_ml/connectors/_create'
    url = host + path

    payload = {
        "name": "Amazon Bedrock Connector: embedding",
        "description": "The connector to bedrock Titan text embedding model",
        "version": 1,
        "protocol": "aws_sigv4",
        "credential": {
          "roleArn": f"arn:aws:iam::{account_id}:role/{bedrock_inf_iam_role}"
       },
       "parameters": {
        "region": region,
        "service_name": "bedrock",
           ## USING AMAZON BEDROCK TITAN EMBED TEXT MODEL
        "model": "amazon.titan-embed-text-v2:0"
       },
       "actions": [
        {
          "action_type": "predict",
          "method": "POST",
          "url": "https://bedrock-runtime.${parameters.region}.amazonaws.com/model/${parameters.model}/invoke",
          "headers": {
            "content-type": "application/json",
            "x-amz-content-sha256": "required"
          },
         "request_body": "{ \"inputText\": \"${parameters.inputText}\" }",
         "pre_process_function": "connector.pre_process.bedrock.embedding",
         "post_process_function": "connector.post_process.bedrock.embedding"}
       ]
    }

    r = requests.post(url, auth=awsauth, json=payload, headers=headers)
    print(r.status_code)
    if r.status_code == 403:
        print("Permission Error: Please make sure you have mapped the NB role to ml_full_access role in OpenSearch dashboard. Follow permission section in lab instructions.")
        print(r.text)
    else:
        connector_id = json.loads(r.text)["connector_id"]
        print(r.text)
else:
    print(f"Connector already exists - {connector_id}")
    
connector_id

Once the model connector is defined. We need to register the model and deploy. Following two cells will register and then deploy the model connection respectively.

In [ ]:
# Register the model
if not model_id:
    path = '_plugins/_ml/models/_register'
    url = 'https://'+aos_host + '/' + path
    payload = { "name": "Bedrock Titan embeddings model",
    "function_name": "remote",
    "description": "Bedrock Titan text embeddings model",
    "connector_id": connector_id}
    r = requests.post(url, auth=awsauth, json=payload, headers=headers)
    model_id = json.loads(r.text)["model_id"]
else:
    print("skipping model registration - model already exists")
print("Model registered under model_id: "+model_id)


finally we will deploy the model which will enable the remote inference. This model is what we will use to conver text to embeddings.

In [ ]:
# Deploy the model
path = '_plugins/_ml/models/'+model_id+'/_deploy'
url = 'https://'+aos_host + '/' + path
r = requests.post(url, auth=awsauth, headers=headers)
deploy_status = json.loads(r.text)["status"]
print("Deployment status of the model, "+model_id+" : "+deploy_status)

#### Create a test embedding to see model that is deployed is working fine.

Lets use `ml_client.generate_embedding` method which is a method in OpenSearch ML Commons plugin python client to call an embedding model to generate embeddings.

**Import pre-requisite**: Please make sure you have followed _**Provision Amazon Bedrock model access**_ section in the lab instruction to setup Amazon Bedrock model access. If you have not done so, you may get an Authorization Exception when you run the following cell.

In [ ]:
#Testing model working

input_sentences = ["What an easy way to create embeddings"]
embedding_output = ml_client.generate_embedding(f"{model_id}", input_sentences)
embed = embedding_output['inference_results'][0]['output'][0]['data']
print(embed[:5])

## 5. Create ingest pipeline
Let's create an ingestion pipeline that will call Amazon Bedrock Titan Text embedding model and convert the `doc_chunk_text` field of the text chunk record to vector embedding. Ingest pipeline is a feature in OpenSearch that allows you to define certain actions to be performed at the time of data ingestion. You could do simple processing such as adding a static field, modify an existing field, or call a remote model to get inference and store inference output together with the indexed record/document. In our case inference output is vector embedding.

Following ingestion pipeline is going to call our remote model and convert chunk text `doc_chunk_text` field to vector and store it in the field called `doc_chunk_embedding`
 

In [ ]:
path =  "/_ingest/pipeline/amazon-report-ingest-pipeline"
url = f"{aos_host}{path}"

payload = {
  "description": "An Index of Amazon annual report",
  "processors": [
    {
      "text_embedding": {
        "model_id": f"{model_id}",
        "field_map": {
          "doc_chunk_text": "doc_chunk_embedding"
        }
      }
    }
  ]
}

aos_client.ingest.put_pipeline(id="amazon-report-ingest-pipeline", body=payload)



## 6. Create a index in Amazon Opensearch Service 
Now we will define our index for our PDF document knowledgebase. We are going to define 2 fields, first text chunk and second has the vector embeddings. For vector embedding, we will use `FAISS` engine and `HNSW` as our algorithm/method. We will use some reasonable defaults for other parameters. Notice that we provide the above created pipeline as the default pipeline for the data ingested into index. This will make the documents go through pipeline before being ingested in the index.

To create the index, we first define the index in JSON, then use the aos_client connection we initiated ealier to create the index in OpenSearch.

In [ ]:
##DEFINE INDEX JSON
knn_index = {
    "settings": {
        "index.knn": True,
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "default_pipeline": "amazon-report-ingest-pipeline", 
        "analysis": {
          "analyzer": {
            "default": {
              "type": "standard",
              "stopwords": "_english_"
            }
          }
        }
    },
    "mappings": {
        "properties": {
             "doc_chunk_text": {
                "type": "text",
                "store": True
            },
           "doc_chunk_embedding": {
               "type": "knn_vector",
               "dimension": 1024,
               "method": {
                   "name": "hnsw",
                   "space_type": "l2",
                   "engine": "faiss",
                   "parameters": {
                       "ef_construction": 256,
                       "m": 48
                   }
               }
           }
        }
    }
}


Using the above index definition, we now need to create the index in Amazon OpenSearch Service. Running this cell will recreate the index if you have already executed this notebook before.

In [ ]:
index_name = "amazon_report_knowledge_base"

try:
    aos_client.indices.delete(index=index_name)
    print("Recreating index '" + index_name + "' on cluster.")
    aos_client.indices.create(index=index_name,body=knn_index,ignore=400)
except:
    print("Index '" + index_name + "' not found. Creating index on cluster.")
    aos_client.indices.create(index=index_name,body=knn_index,ignore=400)


Let's verify the created index information

In [ ]:
aos_client.indices.get(index=index_name)

## 7. Load the raw data into the Index
Next, let's load the chunked text data and its embeddings into the index that we've just created. Notice that we will store our embedding in `doc_chunk_embedding` field which will later be used for KNN search

In [ ]:
cnt = 0
batch = 0
action = json.dumps({ "index": { "_index": index_name } })
body_ = ''


with alive_bar(len(docs), force_tty = True) as bar:
    for doc in docs:

        payload={
           "doc_chunk_text": doc.page_content
        }
        body_ = body_ + action + "\n" + json.dumps(payload) + "\n"
        cnt = cnt+1
        
        if(cnt == 100):
            
            response = aos_client.bulk(
                                index = index_name,
                                 body = body_)
            

            cnt = 0
            batch = batch +1
            body_ = ''
        
        bar()
print("Total Bulk batches completed: "+str(batch))

To validate the load, we'll query the number of documents number in the index. We should have close to 400 documents in the index.

In [ ]:
res = aos_client.search(index=index_name, body={"query": {"match_all": {}}})
print("Records found: %d." % res['hits']['total']['value'])

## 8. Search vector with "Semantic Search" 

Now we can define a helper function to execute the search query for us to find a chunk which matches the user's question semantically. `retrieve_opensearch_with_semantic_search` uses neural query which takes the query text and the model id that it will use to convert the query text into embedding before running a approximate neighbour search i.e. semantic search. We pass K as the parameter to tell how many results we want openseach to return.


In [ ]:
def retrieve_opensearch_with_semantic_search(phrase, n=3):
    osquery={
        "_source": {
            "exclude": [ "doc_chunk_embedding" ]
        },
        
      "size": n,
      "query": {
        "neural": {
          "doc_chunk_embedding": {
            "query_text": f"{phrase}",
            "model_id": f"{model_id}",
            "k": n
          }
        }
      }    
    }

    res = aos_client.search(index=index_name, 
                           body=osquery,
                           stored_fields=["doc_chunk_text"],
                           explain = False)
    top_result = res['hits']['hits']
    
    results = []
    
    for entry in top_result:
        result = {
            "doc_chunk_text":entry['_source']['doc_chunk_text']
           
        }
        results.append(result)
    
    return results


### Use the semantic search to get similar records with the sample question.

We will ask a question that we think can be answered from our chunked data. OpenSearch will return top results that match the question. You will notice the chunks returned do talk about the topics in the question

In [ ]:
question_on_docs = "What was the operating income difference between 2022 and 2023?"
example_request = retrieve_opensearch_with_semantic_search(question_on_docs)
print(json.dumps(example_request, indent=4))

## 9. Prepare a method to call Amazon Bedrock - Anthropic Claude Sonnet model
Now we will define a function to call LLM to answer user's question. As LLM is trained with general purpose data, likely before the PDF was produced. It may not have the knowledge hidden in the PDF file. While it may be able to answer, it may not be an answer that your business prefers. So the answer has to reference the data that we passed in the prompt. 

After defining this function we will call it to see how LLM answers questions from the PDF chunk data.

In [ ]:
def query_llm_endpoint_with_json_payload(encoded_json):

    # Create a Bedrock Runtime client
    bedrock_client = boto3.client('bedrock-runtime')
    # Set the model ID for Claude 3 Sonnet
    model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
    accept = 'application/json'
    content_type = 'application/json'


    try:
        # Invoke the model with the native request payload
        response = bedrock_client.invoke_model(
            modelId=model_id,
            body=str.encode(str(encoded_json)),
            accept = accept,
            contentType=content_type
        )

        # Decode the response body
        response_body = json.loads(response.get('body').read())
        return response_body
    except Exception as e:
        print(f"Error: {e}")
        return none

def query_llm(system, user_question):

    # Prepare the model's payload
    payload = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 10000,
        "system": system,
        "messages": [
            {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": f"{user_question}"
                }
              ]
            }
          ]
        })
    


    query_response = query_llm_endpoint_with_json_payload(payload)

    return query_response['content'][0]['text']


## 10. Retrieval Augmented Generation

#### Create a prompt for the LLM using the search results from OpenSearch (RAG)

We will be using the Anthropic Sonnet model with one-shot prompting technique. Within instructions to the model in the prompt. At the end of the prompt we will provide the chunks retrieved from Opensearch for model to refer to, to answer user's questions. 

Before querying the model, the below function `generate_rag_based_system_prompt` is used to put together user prompt. The function takes in an input string to search the OpenSearch cluster for a matching chunk, then composes the user prompt for LLM. 

System prompt defines the role that LLM plays.

User prompt contains the instructions and the context information that LLM model uses to answer user's question.

The prompt is in the following format:

**SYSTEM PROMPT:**

```
You are a Financial report analysis bot analyzes provided text from financial documents and answers user questions. 
```


**USER PROMPT**
```
As a financial report analyst please answer the question from the provided DOCS_DATA. If you cannot find answer from the DOCS_DATA, please say I'm sorry I cannot answer this question from given information. You do not have to mention that you got answer from DOCS_DATA if you got answer from DOCS_DATA.

Following is the DOCS_DATA after which you will be given the user's question to answer.

DOCS_DATA: {retrieved_documents}

User's Question: <USER QUESTION>
```

We define a method `query_llm_with_rag` that combines everything we've done in this module. It does all of the following:
- searches the OpenSearch index with semantic search for the relevant chunk.
- generate an LLM prompt from the search results
- queriy the LLM with RAG for a response


In [ ]:
def query_llm_with_rag(user_question, n=3):
    retrieved_documents = retrieve_opensearch_with_semantic_search(user_question,n)
    system_prompt= "You are a Financial report analysis bot analyzes provided text from financial documents and answers user questions"
    user_prompt = (
        f"As a financial report analyst please answer the question from the provided DOCS_DATA. If you cannot find answer from the DOCS_DATA, please say I'm sorry I cannot answer this question from given information. You do not have to mention that you got answer from DOCS_DATA if you got answer from DOCS_DATA.\n"
        f"Following is the DOCS_DATA after which you will be given the user's question to answer\n"
        f"DOCS_DATA: {retrieved_documents} \n"
        f"User's Question: {user_question} \n"        
    )
    response = query_llm(system_prompt, user_prompt)
    return response



### Answer questions using RAG
Lets ask a question that can be answered from the PDF text data. We encourage you to change the question to other financial or general company performance related questions which could possibly be answered from the annual report. You will notice that this simple architecture can provide you really good way to analyze a complex document like the annual financial report.

In [ ]:
question_on_docs ="What was the revenue for amazon advertisement in 2023?"
recommendation = query_llm_with_rag(question_on_docs)
print(recommendation)

print(f"\n\ndocuments retrieved for above recommendations were \n\n{json.dumps(retrieve_opensearch_with_semantic_search(question_on_docs), indent=4)}")

# 11. Hybrid search - Advanced RAG
following section will explore how Hybrid search may help in certain situations. 

In Amazon report **knowledge bases** is mentioned as the technology based on RAG or retrieval augmented generation to answer real time queries.

Let's use following question as an example to search for AWS offering for RAG. The semantic meaning of RAG may not be known to the embedding model, which is why it would use other terms in the user's query to match information in the chunks. This may result in LLM not able to answer our question. 


In [ ]:
question_on_docs ="What is AWS offering for retrieval augmented generation?"
recommendation = query_llm_with_rag(question_on_docs, n=3)
print(recommendation)

print(f"\n\ndocuments retrieved for above recommendations were \n\n{json.dumps(retrieve_opensearch_with_semantic_search(question_on_docs, n=3), indent=4)}")

### How do we solve this?
As you can see the top 3 chunks do not contain the information that could answer user's question. In such cases vector search may find a number of similar looking chunks, however, just by looking at similarity distance, it may not return the chunk that has the answer, to the top. We could augment semantic search with traditional keyword search and combine the results. This is called **Hybrid search**. The results from both keywords and semantic search are merged with various normalization and combination techniques to produce final results. Let us now define a search pipeline that uses **_phase_results_processors_** results processor that takes 2 different queries and combine their results. In our case we will give 40% weightage to keyword matches and 60% to semantic matches, and results are combined using **harmonic mean** technique. 

Our queries will be a full text search and a semantic search as before. We rewrite our retrieval method and add keyword query, we call this method _retrieve_opensearch_with_hybrid_search( )_, notice how this method has 2 query clauses under "**hybrid**" section

In [ ]:
path =  "/_search/pipeline/hybrid-search-pipeline"
url = f"https://{aos_host}{path}"

payload = {
  "description": "Hybrid search over amazon financial report",
  "phase_results_processors":[
      {
          "normalization-processor":{
              "normalization": {
                  "technique": "l2"
              },
              "combination": {
                  "technique": "harmonic_mean",
                  "parameters": {
                      "weights": [
                          0.4, #first query i.e. keywords have 20% weightage
                          0.6  #first query i.e. semantic search have 80% weightage
                      ]
                  }
              }
          }
      }
    ]
}

r = requests.put(url, auth=awsauth, json=payload, headers=headers)
print(r.status_code)
print(r.text)



def retrieve_opensearch_with_hybrid_search(phrase, model_id=model_id, bedrock_client=bedrock_client, n=3 ):
    osquery={
            "_source": {
                "exclude": [ "doc_chunk_embedding" ]
            },
          "size": n,
          "query": {
            "hybrid": {
              "queries": [
                {
                  #First query clause that performs keyword search
                  "match": {
                    "doc_chunk_text": {
                      "query": f"{phrase}"
                    }
                  }
                },
                {
                  #Second query clause that performs semantic search
                  "neural": {
                    "doc_chunk_embedding": {
                      "query_text": f"{phrase}",
                      "model_id": f"{model_id}",
                      "k": n
                    }
                  }
                }
              ]
            }
          }    
        }

    res = aos_client.search(index=index_name, 
                           body=osquery,
                           search_pipeline="hybrid-search-pipeline",
                           stored_fields=["doc_chunk_text"],
                           explain = False)
    top_result = res['hits']['hits']
    
    results = []
    
    for entry in top_result:
        result = {
            "id":entry['_id'],
            "doc_chunk_text":entry['_source']['doc_chunk_text'],
            "_score":entry['_score']
           
        }
        results.append(result)
    
    return results


def hybrid_query_llm_with_rag(user_question):
    retrieved_documents = retrieve_opensearch_with_hybrid_search(user_question)
    system_prompt= "You are a Financial report analysis bot analyzes provided text from financial documents and answers user questions"
    user_prompt = (
        f"As a financial report analyst please answer the question from the provided DOCS_DATA. If you cannot find answer from the DOCS_DATA, please say I'm sorry I cannot answer this question from given information. You do not have to mention that you got answer from DOCS_DATA if you got answer from DOCS_DATA.\n"
        f"Following is the DOCS_DATA after which you will be given the user's question to answer\n"
        f"DOCS_DATA: {retrieved_documents} \n"
        f"User's Question: {user_question} \n"        
    )
    response = query_llm(system_prompt, user_prompt)
    return response

In [ ]:
question_on_docs ="What is AWS offering for retrieval augmented generation?"
recommendation = hybrid_query_llm_with_rag(question_on_docs)
print(recommendation)

print(f"\n\ndocuments retrieved for above recommendations were \n\n{json.dumps(retrieve_opensearch_with_hybrid_search(question_on_docs, model_id=model_id, bedrock_client=bedrock_client), indent=4)}")

### Hybrid search is popular but...
We are seeing customers use hybrid search often to find relevant results when only semantic search may not be the only solution. This highly applicable if your use case has domain specific terminologies (e.g. RAG) that embedding model may not have seen enough in training data or its very specific to your company taxonomy and might mean very differently to that of its traditional meaning.

It could also be that the top 3 results that semantic search found do not have the answer. Let's try semantic search again, this time with **n=10** i.e. top 10 results.

In [ ]:
question_on_docs ="What is AWS offering for retrieval augmented generation?"

#Setting n=10 to retrieve 10 results instead of 3
n = 10
recommendation = query_llm_with_rag(question_on_docs, n)
print(recommendation)

print(f"\n\ndocuments retrieved for above recommendations were \n\n{json.dumps(retrieve_opensearch_with_semantic_search(question_on_docs, n=n), indent=4)}")

As you can see model could answer the question a little better.

## 12. Re-ranking using a cross encoder model 

Above problem can be solved if we can find a way to rerank results in such as a way that text chunk that talks about RAG becomes within the top 3 results. This could potentially be achieved with cross encoder model which help us rerank opensearch results by looking at the query and result sets.

Cross encoder models are supported in OpenSearch and two of such models are available in OpenSearch service.

Cross encoder model look at the query and search results and rerank them in such a way that results that could possibly answer the query would be ranked higher. 

### Lets set some cluster setting that enable local model hosting

In [ ]:
s = b"""
{"transient":{"plugins.ml_commons.only_run_on_ml_node": false}}
"""
aos_client.cluster.put_settings(body=s)

## Deploy cross encoder model
We will use ML commons feature in opensearch to load a cross encoder model. This model is hosted within opensearch in a data node. Earlier we used ML commons to deploy a remote inference model that call Amazon Bedrock Titan. This time we will register and deploy a cross encoder model within a data node.

In [ ]:
model_group_id=""

path =  "/_plugins/_ml/model_groups/_register"
url = f"https://{aos_host}{path}"

payload = {
  "name": "local_model_group",
  "description": "A model group for local models"
}

r = requests.post(url, auth=awsauth, json=payload, headers=headers)
print(r.status_code)
print(r.text)
model_group_id = json.loads(r.text)["model_group_id"]

Now we will register a model ..

In [ ]:
path =  "/_plugins/_ml/models/_register"
url = f"https://{aos_host}{path}"

payload = {
  "name": "huggingface/cross-encoders/ms-marco-MiniLM-L-6-v2",
  "version": "1.0.2",
  "model_group_id": f"{model_group_id}",
  "model_format": "TORCH_SCRIPT"
}

r = requests.post(url, auth=awsauth, json=payload, headers=headers)
print(r.status_code)
print(r.text)
rr_model_task_id = json.loads(r.text)["task_id"]

rr_model_task_id

Following cell will check if the model registration is completed. Please run it until you see model registered 

In [ ]:
rr_model_id = ""

path =  f"/_plugins/_ml/tasks/{rr_model_task_id}"
url = f"https://{aos_host}{path}"

r = requests.get(url, auth=awsauth, headers=headers)
print(r.status_code)
print(r.text)
task_state = json.loads(r.text)["state"]
if task_state == "COMPLETED":
    rr_model_id = json.loads(r.text)["model_id"]
    print("TASK COMPLETED SUCCESSFULLY, PLEASE MOVE TO NEXT CELL")
else:
    print("TASK NOT COMPLETED, PLEASE RE-RUN THIS CELL")


Following code will deploy the registered model

In [ ]:
path =  f"/_plugins/_ml/models/{rr_model_id}/_deploy"
url = f"https://{aos_host}{path}"

r = requests.post(url, auth=awsauth, json=payload, headers=headers)
print(r.status_code)
print(r.text)
rr_model_task_id = json.loads(r.text)["task_id"]



In [ ]:
path =  f"/_plugins/_ml/tasks/{rr_model_task_id}"
url = f"https://{aos_host}{path}"

r = requests.get(url, auth=awsauth, headers=headers)
print(r.status_code)
print(r.text)
task_state = json.loads(r.text)["state"]
if task_state == "COMPLETED":
    print("MODEL DEPLOYMENT SUCCESSFUL, PLEASE MOVE TO THE NEXT CELL")
    rr_model_id = json.loads(r.text)["model_id"]
elif task_state == "FAILED":
    print("MODEL DEPLOYMENT FAILED, PLEASE INVESTIGATE THE ERROR")
else:
    print("MODEL DEPLOYMENT IN PROGRESS, PLEASE RE-RUN THE CELL")

### Create a search pipeline that uses re-ranker model 
Now we will deploy a search pipeline that will pass the search results to reranker model that we just deployed. The model requires names of the field that cross encoder will use to assess result ranking. In this case we provide `doc_chunk_text` as the field.

In [ ]:
path =  "/_search/pipeline/rerank-search-pipeline"
url = f"https://{aos_host}{path}"

payload = {
  "description": "Pipeline for reranking with cross-encoder model",
    "response_processors": [
        {
            "rerank": {
                "ml_opensearch": {
                    "model_id": f"{rr_model_id}"
                },
                "context": {
                    "document_fields": ["doc_chunk_text"]
                }
            }
        }
    ]
}

print(payload)

r = requests.put(url, auth=awsauth, json=payload, headers=headers)
print(r.status_code)
print(r.text)

Lets rewrite our semantic search method, we will use rerank clause and provide our query as the query context. With this query and the `doc_chunk_text` we provided as part of search pipeline, cross encoder has everything it needs perform the reranking. We will define following two methods

`retrieve_opensearch_with_semantic_rerank_search` - Method runs a semantic search uses rerank clause to rerank results. When it runs search, it uses `rerank-search-pipeline` as the search pipeline.

`rerank_query_llm_with_rag` - This method calls rerank search method above to retrieve top chunks and then passes the top chunks to Claude Sonnet 3 to answer user questions.

In [ ]:
def retrieve_opensearch_with_semantic_rerank_search(phrase, n, k, model_id=model_id, bedrock_client=bedrock_client):
    osquery={
        "_source": {
            "exclude": [ "doc_chunk_embedding" ]
        },
        
      "size": k,
      "query": {
        "neural": {
          "doc_chunk_embedding": {
            "query_text": f"{phrase}",
            "model_id": f"{model_id}",
            "k": k
          }
        }
      },
     "ext":{
         "rerank": {
          "query_context": {
             "query_text": f"{phrase}"
        }
     }
        
    }
    }

    res = aos_client.search(index=index_name, 
                           body=osquery,
                           search_pipeline="rerank-search-pipeline",
                           stored_fields=["doc_chunk_text"],
                           explain = False)
    top_result = res['hits']['hits']

    results = []
    for entry in top_result:
        result = {
            "id":entry['_id'],
            "doc_chunk_text":entry['_source']['doc_chunk_text'],
            "_score":entry['_score']
           
        }
        results.append(result)

    return results[:n]


def rerank_query_llm_with_rag(user_question, n, k):
    retrieved_documents = retrieve_opensearch_with_semantic_rerank_search(user_question, n = n, k = k, model_id=model_id, bedrock_client=bedrock_client)
    system_prompt= "You are a Financial report analysis bot analyzes provided text from financial documents and answers user questions"
    user_prompt = (
        f"As a financial report analyst please answer the question from the provided DOCS_DATA. If you cannot find answer from the DOCS_DATA, please say I'm sorry I cannot answer this question from given information. You do not have to mention that you got answer from DOCS_DATA if you got answer from DOCS_DATA.\n"
        f"Following is the DOCS_DATA after which you will be given the user's question to answer\n"
        f"DOCS_DATA: {retrieved_documents} \n"
        f"User's Question: {user_question} \n"        
    )
    response = query_llm(system_prompt, user_prompt)
    return response

## Run cross encoder search
Following cell runs a search for the same question that could not have been answered. We will fetch K = 10 items, which will be reranked by cross encoder model. From the top K we will pass top N records to the LLM for answering user's question.

In [ ]:
question_on_docs ="What is AWS offering for retrieval augmented generation?"

#keeping our results to top 3 items.
n = 3
recommendation = rerank_query_llm_with_rag(question_on_docs, n=3, k=10)
print(recommendation)

print(f"\n\ndocuments retrieved for above recommendations were \n\n{json.dumps(retrieve_opensearch_with_semantic_rerank_search(phrase=question_on_docs, n=3, k=10), indent=4)}")

# Conclusion
You started by loading and parsing Amazon annual financial report in PDF file format, chunking the loaded text and using Neural plugin converted chunks into embedding without having to handle the embeddings yourself. Then you used semantic search to answer user's finance related question over this data. We saw how hybrid search can improve our retrieval by combining powerful keyword with novel semantic search capability. We also showed how reranking can be an important enhancement in your RAG pipeline.

Congratulations on finsihing this lab. You may now go to the lab instructions section.
